In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg
from mv_wg_avg import exp_weight_avg

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'valid_no_2', 'is_train', 'is_test', 'SK_ID_PREV']

2018-07-08 16:33:11,321 logger 19 [INFO]    [logger_func] start 
2018-07-08 16:33:11,339 logger 19 [INFO]    [logger_func] start 
2018-07-08 16:33:11,339 logger 19 [INFO]    [logger_func] start 


In [2]:
base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
prev = pd.read_csv('../data/previous_cleansing.csv')
app = pd.read_csv('../data/application_train_test.csv')

' applicationとmerge '
ext_avg = pd.Series(np.load('../features/3_winner/a_EXT_SOURCE_avg@.npy'), name='a_EXT_SOURCE_avg@')
app['a_EXT_SOURCE_avg@'] = ext_avg
app_list = [unique_id, 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'a_EXT_SOURCE_avg@']
app = app[app_list]

for col in app.columns:
    if col.count('AMT'):
        app.rename(columns = {col:f'app_{col}'}, inplace=True)
        
prev_app = prev.merge(app, on=unique_id, how='inner')

In [3]:
level = unique_id
method_list = [
    'sum',
    'mean',
    'max',
    'min',
    'std'
]
days_list = [
    'DAYS_DECISION',
    'DAYS_FIRST_DRAWING',
    'DAYS_FIRST_DUE',
    'DAYS_LAST_DUE_1ST_VERSION',
    'DAYS_LAST_DUE',
    'DAYS_TERMINATION',
    'DAYS_lastplan_month',
    'DAYS_original_last_due',
    'DAYS_LAST_DUE_diff_lastplan_month',
    'DAYS_term_month',
    'DAYS_term_month_1st'
]
amt_list = [
    'AMT_ANNUITY',
    'AMT_APPLICATION',
    'AMT_CREDIT',
    'AMT_DOWN_PAYMENT',
    'cnt_annuity'
]
elem_list = [
    'CNT_PAYMENT' ,
    'DAYS_term_month',
    'cnt_annuity',
    'DAYS_term_month_1st',
]

In [4]:
' 使うfeatureを先に作成しておく '
dataset = prev_app

" last_due - (decision + (amt_credit_div_amt_annuity)) 実際の完了日と支払額から月毎の返済として逆算した場合の月数の差 "
dataset['DAYS_lastplan_month'] =  dataset[days_list[0]] + ( dataset[amt_list[2]] / dataset[amt_list[0]] ) 
dataset['DAYS_LAST_DUE_diff_lastplan_month'] = dataset[days_list[4]] - dataset['DAYS_lastplan_month']
dataset['DAYS_original_last_due'] = dataset[days_list[0:7]].apply(lambda x: x[4] if x[4]==x[4] else x[3] if x[3]==x[3] else x[6], axis=1)

' first_due - decision '
dataset = diff_feature(df=dataset, first=days_list[0], second=days_list[2])
' last_due - due_1st_version '
dataset = diff_feature(df=dataset, first=days_list[3], second=days_list[4])
' termination - last_due '
dataset = diff_feature(df=dataset, first=days_list[4], second=days_list[5])
' decision - last_due '
dataset['DAYS_term_month'] = dataset[days_list[4]] - dataset[days_list[0]]
' decision - last_due_1st '
dataset['DAYS_term_month_1st'] = dataset[days_list[3]] - dataset[days_list[0]]

' credit / application '
dataset = division_feature(df=dataset, first=amt_list[2], second=amt_list[1])
' credit / annuity '
dataset['cnt_annuity'] = dataset[amt_list[2]] / dataset[amt_list[0]]
' cnt_annuity - cnt_payment  '
dataset = diff_feature(df=dataset, first=elem_list[2], second=elem_list[0])

' amt_credit / term_month 月あたり支払額⇒予定より早く終了してる人が多くいるから'
dataset['AMT_CREDIT_div_term_month'] = dataset[amt_list[2]] / dataset[amt_list[1]]
' amt_credit / term_month_1st 月あたり支払額'
dataset['AMT_CREDIT_div_term_month_1st'] = dataset[amt_list[2]] / dataset[amt_list[3]]

In [5]:
' applicationとの組み合わせfeatureを先に作成しておく '
dataset = prev_app

dataset = division_feature(df=dataset, first='app_AMT_INCOME_TOTAL', second='AMT_CREDIT')
dataset = division_feature(df=dataset, first='app_AMT_INCOME_TOTAL', second='AMT_APPLICATION')
dataset = division_feature(df=dataset, first='app_AMT_INCOME_TOTAL', second='AMT_ANNUITY')

In [6]:
dataset = prev_app
status = 'NAME_CONTRACT_STATUS'

' Approveされたデータ '
approve = dataset.query("NAME_CONTRACT_STATUS=='Approved'")
' Refuseされたデータ '
refuse = dataset.query("NAME_CONTRACT_STATUS=='Refused'")
cancel = dataset.query("NAME_CONTRACT_STATUS=='Canceled'")
unuse = dataset.query("NAME_CONTRACT_STATUS=='Unused offer'")

In [14]:
category = 'CHANNEL_TYPE'
a = prev_app.query('TARGET!=-1')[[unique_id, target, category]].drop_duplicates().groupby([category])[target].mean().sort_values(ascending=False)
b = prev_app[category].value_counts()
c = pd.concat([a, b], axis=1).sort_values(by=target, ascending=False)
display(c)

,TARGET,CHANNEL_TYPE
AP+ (Cash loan),0.112792,57046
Contact center,0.095658,71297
Credit and cash offices,0.083328,719968
Country-wide,0.081940,494690
Regional / Local,0.076057,108528
Stone,0.074257,212083
Channel of corporate sales,0.057130,6150
Car dealer,0.047170,452


In [39]:
prev_app['LOAN_RISK_TYPE'] = prev_app['NAME_CASH_LOAN_PURPOSE'].map(risk_type)
prev_app['REJECT_TYPE'] = prev_app['CODE_REJECT_REASON'].map(lambda x: 'XAP' if x=='SYSTEM' else 'XNA' if x=='VERIF' else x)

In [106]:
' 粒度を指定して集計する '
prefix = 'p_APR_YEAR_'
# prefix = 'p_NEW_YEAR_'
level = [unique_id]
data = apr_year
feature_list = ['SELLERPLACE_AREA']
select_level_agg(base=base, data=data, level=level, num_list=feature_list, method_list=method_list, ignore_list=ignore_features, prefix=prefix)

2018-07-06 21:51:34,897 logger 125 [INFO]    [select_level_agg] 
num: SELLERPLACE_AREA
method: sum
['SK_ID_CURR'] 
2018-07-06 21:51:34,897 logger 125 [INFO]    [select_level_agg] 
num: SELLERPLACE_AREA
method: sum
['SK_ID_CURR'] 
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
2018-07-06 21:51:34,954 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SELLERPLACE_AREA_sum@['SK_ID_CURR']'], dtype='object') 
2018-07-06 21:51:34,954 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SELLERPLACE_AREA_sum@['SK_ID_CURR']'], dtype='object') 
2018-07-06 21:51:35,038 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 21:51:35,038 logger 132 [INFO]    [select_level_agg] 
re

2018-07-06 21:51:35,252 logger 125 [INFO]    [select_level_agg] 
num: SELLERPLACE_AREA
method: max
['SK_ID_CURR'] 
2018-07-06 21:51:35,252 logger 125 [INFO]    [select_level_agg] 
num: SELLERPLACE_AREA
method: max
['SK_ID_CURR'] 
2018-07-06 21:51:35,297 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SELLERPLACE_AREA_max@['SK_ID_CURR']'], dtype='object') 
2018-07-06 21:51:35,297 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SELLERPLACE_AREA_max@['SK_ID_CURR']'], dtype='object') 
2018-07-06 21:51:35,366 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 21:51:35,366 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 21:51:35,374 logger 29 [INFO]    [make_npy] 100.0     7124
50.0      6882
30.0      4750
20.0      4526
40.0      3501
150.0     2875
200.0     2821
0.0       2468
15.0      2019
25.0  

2018-07-06 21:51:35,596 logger 125 [INFO]    [select_level_agg] 
num: SELLERPLACE_AREA
method: std
['SK_ID_CURR'] 
2018-07-06 21:51:35,653 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SELLERPLACE_AREA_std@['SK_ID_CURR']'], dtype='object') 
2018-07-06 21:51:35,653 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'SELLERPLACE_AREA_std@['SK_ID_CURR']'], dtype='object') 
2018-07-06 21:51:35,723 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 21:51:35,723 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 21:51:35,729 logger 29 [INFO]    [make_npy] 0.000000       12497
7.071068         303
14.142136        277
35.355339        266
21.213203        246
3.535534         183
10.606602        167
28.284271        154
17.677670        134
70.710678        133
49.497475        121
24.748737        104
56.5

In [ ]:
prefix = 'p_'
dataset = prev_app
' 重み付き平均 '
weight_list = [0.99, 0.97, 0.95, 0.9]
feature_list = amt_list
# feature_list =['AMT_APPLICATION_div_app_AMT_INCOME_TOTAL@', 'AMT_CREDIT_div_app_AMT_INCOME_TOTAL@', 'app_AMT_INCOME_TOTAL_div_AMT_ANNUITY@' ]
for num in feature_list:
    for weight in weight_list:
        prev_wavg = exp_weight_avg(data=dataset, level=unique_id, weight=weight, label='DAYS_DECISION', value=num)
        result = base.merge(prev_wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
        make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)

In [42]:
approve.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,TARGET,valid_no,is_train,is_test,diff,weight
0,2462805,299809,Cash loans,9842.400,90000.0,90000.0,NaN,90000.0,THURSDAY,13,Y,1.0,NaN,NaN,NaN,XNA,Approved,-75.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash Street: high,-75.0,-74.0,-63.0,-63.0,-63.0,0.0,0,3.0,1,0,75.0,0.470587
1,1626275,426902,Cash loans,5246.010,45000.0,47970.0,NaN,45000.0,THURSDAY,15,Y,1.0,NaN,NaN,NaN,XNA,Approved,-74.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash Street: high,-74.0,-73.0,-62.0,-62.0,-62.0,1.0,0,3.0,1,0,74.0,0.475340
2,1983905,157753,Cash loans,8610.120,112500.0,127350.0,NaN,112500.0,THURSDAY,14,Y,1.0,NaN,NaN,NaN,XNA,Approved,-74.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,24.0,high,Cash Street: high,-74.0,-73.0,-50.0,-50.0,-50.0,1.0,0,4.0,1,0,74.0,0.475340
3,2462807,343731,Cash loans,17887.500,225000.0,225000.0,NaN,225000.0,THURSDAY,12,Y,1.0,NaN,NaN,NaN,XNA,Approved,-77.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,18.0,middle,Cash Street: middle,-77.0,-76.0,-59.0,-59.0,-59.0,0.0,0,2.0,1,0,77.0,0.461222
4,2006470,429750,Cash loans,19681.965,270000.0,299938.5,NaN,270000.0,THURSDAY,18,Y,1.0,NaN,NaN,NaN,XNA,Approved,-80.0,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,24.0,middle,Cash Street: middle,-84.0,-83.0,-60.0,-64.0,-64.0,1.0,-1,-1.0,0,1,80.0,0.447523


In [93]:
dataset = cash_halfyear

" last_due - (decision + (amt_credit_div_amt_annuity)) 実際の完了日と支払額から月毎の返済として逆算した場合の月数の差 "
dataset['DAYS_lastplan_month'] =  dataset[days_list[0]] + ( dataset[amt_list[2]] / dataset[amt_list[0]] ) 
dataset['DAYS_LAST_DUE_diff_lastplan_month'] = dataset[days_list[4]] - dataset['DAYS_lastplan_month']
dataset['DAYS_original_last_due'] = dataset[days_list[0:7]].apply(lambda x: x[4] if x[4]==x[4] else x[3] if x[3]==x[3] else x[6], axis=1)

' first_due - decision '
dataset = diff_feature(df=dataset, first=days_list[0], second=days_list[2])
' last_due - due_1st_version '
dataset = diff_feature(df=dataset, first=days_list[3], second=days_list[4])
' termination - last_due '
dataset = diff_feature(df=dataset, first=days_list[4], second=days_list[5])
' decision - last_due '
dataset['DAYS_term_month'] = dataset[days_list[4]] - dataset[days_list[0]]
' decision - last_due_1st '
dataset['DAYS_term_month_1st'] = dataset[days_list[3]] - dataset[days_list[0]]

' credit / application '
dataset = division_feature(df=dataset, first=amt_list[2], second=amt_list[1])
' credit / annuity '
dataset['cnt_annuity'] = dataset[amt_list[2]] / dataset[amt_list[0]]
' cnt_annuity - cnt_payment  '
dataset = diff_feature(df=dataset, first=elem_list[2], second=elem_list[0])

' amt_credit / term_month 月あたり支払額⇒予定より早く終了してる人が多くいるから'
dataset['AMT_CREDIT_div_term_month'] = dataset[amt_list[2]] / dataset[amt_list[1]]
' amt_credit / term_month_1st 月あたり支払額'
dataset['AMT_CREDIT_div_term_month_1st'] = dataset[amt_list[2]] / dataset[amt_list[3]]

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


IndexError: ('index out of bounds', 'occurred at index 997148')

In [91]:
def select_term(df, term):
    
    ' 直近1年に絞る '
    result = df.query(f'DAYS_DECISION >={term}')
    ' credit / application '
    result = division_feature(df=result, first=amt_list[2], second=amt_list[1])
    ' credit / annuity '
    result['cnt_annuity'] = result[amt_list[2]] / result[amt_list[0]]
    
    return result

cash_year = select_term(cash, -12)
cash_halfyear = select_term(cash, -6)

../../../github/module/feature_engineering.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[f'{f1}_div_{f2}@'] = df[f1] / df[f2]
/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
new_ref[[unique_id, 'is_train', target]].query('is_train==1').drop_duplicates().shape

NameError: name 'new_ref' is not defined

In [46]:
'Refuseのfeature'
" last_due - (decision + (amt_credit_div_amt_annuity)) 実際の完了日と支払額から月毎の返済として逆算した場合の月数の差 "
refuse['DAYS_lastplan_month'] =  refuse[days_list[0]] + ( refuse[amt_list[2]] / refuse[amt_list[0]] ) 

# ' credit / application '
# refuse = division_feature(df=refuse, first=amt_list[2], second=amt_list[1])
# ' credit / annuity '
# refuse['cnt_annuity'] = refuse[amt_list[2]] / refuse[amt_list[0]]
# ' cnt_annuity - cnt_payment  '
# refuse = diff_feature(df=refuse, first=elem_list[2], second=elem_list[0])

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
level = unique_id
method_list = [
    'sum',
    'mean',
    'max',
    'min',
    'std'
]
feature_list = [
    'AMT_ANNUITY',
    'AMT_APPLICATION',
    'AMT_CREDIT',
    'AMT_CREDIT_div_AMT_APPLICATION@',
    'cnt_annuity'
#     'CNT_PAYMENT_diff_cnt_annuity@',
]

prefix = 'p_APR_'
prefix = 'p_REF_'
prefix = 'p_REF_HALFYEAR'
prefix = 'p_APR_YEAR'
feature_list = [col for col in refuse.columns if col.count('DAYS_')]
data = approve_year

select_level_agg(base=base, data=data, level=level, num_list=feature_list, method_list=method_list, ignore_list=ignore_features, prefix=prefix)

2018-07-06 08:04:13,932 logger 125 [INFO]    [select_level_agg] 
num: DAYS_DECISION
method: sum
['SK_ID_CURR'] 
2018-07-06 08:04:13,932 logger 125 [INFO]    [select_level_agg] 
num: DAYS_DECISION
method: sum
['SK_ID_CURR'] 
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
2018-07-06 08:04:14,097 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_DECISION_sum@['SK_ID_CURR']'], dtype='object') 
2018-07-06 08:04:14,097 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_DECISION_sum@['SK_ID_CURR']'], dtype='object') 
2018-07-06 08:04:14,172 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 08:04:14,172 logger 132 [INFO]    [select_level_agg] 
result shape: 

2018-07-06 08:04:14,397 logger 125 [INFO]    [select_level_agg] 
num: DAYS_DECISION
method: max
['SK_ID_CURR'] 
2018-07-06 08:04:14,397 logger 125 [INFO]    [select_level_agg] 
num: DAYS_DECISION
method: max
['SK_ID_CURR'] 
2018-07-06 08:04:14,444 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_DECISION_max@['SK_ID_CURR']'], dtype='object') 
2018-07-06 08:04:14,444 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_DECISION_max@['SK_ID_CURR']'], dtype='object') 
2018-07-06 08:04:14,513 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 08:04:14,513 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 08:04:14,523 logger 29 [INFO]    [make_npy] -8.0     14631
-9.0     14468
-10.0    14092
-7.0     14006
-11.0    13980
-6.0     13668
-12.0    12844
-5.0     12093
-4.0     10318
-3.0      8978
 0.

2018-07-06 08:04:15,045 logger 29 [INFO]    [make_npy]  0.0     132161
-7.0       2964
-8.0       2689
-6.0       2630
-9.0       2273
-10.0      2016
-11.0      1682
-5.0       1341
-12.0      1173
-4.0        565
-3.0        487
-1.0        417
-2.0        405
-22.0         2
-17.0         2
-15.0         1
-18.0         1
-19.0         1
-20.0         1
-14.0         1
-16.0         1
Name: DAYS_FIRST_DRAWING_sum@['SK_ID_CURR'], dtype: int64 
2018-07-06 08:04:15,045 logger 29 [INFO]    [make_npy]  0.0     132161
-7.0       2964
-8.0       2689
-6.0       2630
-9.0       2273
-10.0      2016
-11.0      1682
-5.0       1341
-12.0      1173
-4.0        565
-3.0        487
-1.0        417
-2.0        405
-22.0         2
-17.0         2
-15.0         1
-18.0         1
-19.0         1
-20.0         1
-14.0         1
-16.0         1
Name: DAYS_FIRST_DRAWING_sum@['SK_ID_CURR'], dtype: int64 
2018-07-06 08:04:15,080 logger 125 [INFO]    [select_level_agg] 
num: DAYS_FIRST_DRAWING
method: mea

2018-07-06 08:04:16,043 logger 29 [INFO]    [make_npy]  0.0     15260
-10.0    14802
-9.0     14167
-8.0     13692
-11.0    13224
-7.0     12867
-6.0     11621
-5.0     10491
-4.0      9138
-3.0      6844
-2.0      5314
-1.0      4046
-13.0     2397
-12.0     2391
-15.0     2373
-14.0     2349
-16.0     2097
-17.0     1820
-18.0     1527
-19.0     1176
-20.0      910
-21.0      647
-22.0      432
-23.0      242
-25.0      198
-24.0      191
-26.0      135
-27.0      102
-28.0       97
-29.0       64
-30.0       50
-31.0       36
-32.0       30
-33.0       17
-34.0       16
-38.0       10
-35.0        9
-37.0        8
-39.0        6
-43.0        4
-36.0        3
-40.0        3
-42.0        2
-45.0        2
-48.0        1
-44.0        1
-41.0        1
Name: DAYS_FIRST_DUE_sum@['SK_ID_CURR'], dtype: int64 
2018-07-06 08:04:16,096 logger 125 [INFO]    [select_level_agg] 
num: DAYS_FIRST_DUE
method: mean
['SK_ID_CURR'] 
2018-07-06 08:04:16,096 logger 125 [INFO]    [select_level_agg] 
num: D

2018-07-06 08:04:16,730 logger 125 [INFO]    [select_level_agg] 
num: DAYS_FIRST_DUE
method: std
['SK_ID_CURR'] 
2018-07-06 08:04:16,730 logger 125 [INFO]    [select_level_agg] 
num: DAYS_FIRST_DUE
method: std
['SK_ID_CURR'] 
2018-07-06 08:04:16,864 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_FIRST_DUE_std@['SK_ID_CURR']'], dtype='object') 
2018-07-06 08:04:16,864 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'DAYS_FIRST_DUE_std@['SK_ID_CURR']'], dtype='object') 
2018-07-06 08:04:16,983 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 08:04:16,983 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 08:04:16,993 logger 29 [INFO]    [make_npy] 0.707107    3599
1.414214    3360
2.121320    2998
2.828427    2886
3.535534    2847
4.242641    2393
0.000000    1817
4.949747    1752
5.656854    1

...

2018-07-06 08:04:18,781 logger 29 [INFO]    [make_npy] 0.707107    1001
0.000000     997
2.121320     683
1.414214     656
2.828427     615
3.535534     507
4.242641     376
4.949747     226
5.656854     154
6.363961      78
2.645751      37
0.577350      30
1.732051      30
7.071068      30
1.527525      25
2.081666      25
2.309401      20
1.154701      20
7.778175      19
0.577350      19
3.464102      18
2.886751      17
2.516611      17
1.000000      16
3.214550      14
3.214550      14
3.055050      14
2.309401      14
1.527525      13
4.358899      12
4.041452      12
3.605551      11
2.081666      10
3.511885       9
3.785939       9
2.886751       8
1.154701       8
4.000000       7
2.516611       7
3.000000       6
3.511885       6
4.509250       5
2.000000       5
5.196152       5
4.725816       5
3.055050       5
3.785939       5
1.892969       4
4.932883       4
4.163332       4
            ... 
4.082483       2
3.304038       2
4.349329       2
3.162278       2
2.302173  

2018-07-06 08:04:19,280 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 08:04:19,286 logger 29 [INFO]    [make_npy]  0.0     10178
-1.0      8754
-2.0      7673
-3.0      6572
-4.0      6525
-5.0      6195
-6.0      4994
-7.0      3176
-8.0      2289
-9.0      1411
-10.0      812
-11.0      286
-12.0        6
Name: DAYS_TERMINATION_max@['SK_ID_CURR'], dtype: int64 
2018-07-06 08:04:19,286 logger 29 [INFO]    [make_npy]  0.0     10178
-1.0      8754
-2.0      7673
-3.0      6572
-4.0      6525
-5.0      6195
-6.0      4994
-7.0      3176
-8.0      2289
-9.0      1411
-10.0      812
-11.0      286
-12.0        6
Name: DAYS_TERMINATION_max@['SK_ID_CURR'], dtype: int64 
2018-07-06 08:04:19,323 logger 125 [INFO]    [select_level_agg] 
num: DAYS_TERMINATION
method: min
['SK_ID_CURR'] 
2018-07-06 08:04:19,323 logger 125 [INFO]    [select_level_agg] 
num: DAYS_TERMINATION
method: min
['SK_ID_CURR'] 
2018-07-06 08:04:19,364 logger 129 [INFO]    [select_level_agg] 

2018-07-06 08:04:19,793 logger 29 [INFO]    [make_npy]  12.000000    1473
 13.000000    1359
 11.000000    1300
 10.000000    1294
 9.000000     1214
 8.000000     1104
 14.000000    1101
 15.000000     501
 8.250709      434
 9.250709      385
 16.000000     322
 10.250709     313
 17.000000     283
 11.250709     263
 12.250709     262
 20.000000     242
 18.000000     220
 13.250709     174
 19.000000     167
 20.250709     154
 14.250709     151
 15.250709     118
 16.250709     112
-0.909091      111
 18.250709      96
-2.909091       94
-1.909091       86
 8.023588       84
 9.023588       81
 0.090909       75
 17.250709      71
 10.023588      67
 12.023588      62
-2.909091       59
 11.023588      56
 1.090909       55
 19.250709      54
 13.023588      53
 22.250709      52
 0.090909       51
 24.250709      49
-1.909091       49
 3.695187       48
-0.909091       47
-1.241044       44
-2.241044       43
 2.695187       42
-0.241044       42
-0.286480       41
 4.695187     

2018-07-06 08:04:20,170 logger 29 [INFO]    [make_npy]  12.000000    3769
 13.000000    3473
 11.000000    3410
 10.000000    3252
 9.000000     3034
 14.000000    2948
 8.000000     2550
 15.000000    1910
 16.000000    1369
 17.000000    1030
 18.000000     672
 20.000000     577
 8.250709      540
 19.000000     524
 9.250709      516
 10.250709     444
 12.250709     384
 11.250709     377
 13.250709     275
 14.250709     265
 15.250709     212
 20.250709     177
 16.250709     149
 17.250709     126
-0.909091      117
 8.023588      108
 9.023588      106
 18.250709      98
-2.909091       95
 10.023588      93
 11.023588      91
-1.909091       88
 12.023588      85
 0.090909       79
 19.250709      77
 13.023588      75
 4.695187       67
 3.695187       65
 14.023588      64
-2.909091       62
 1.090909       60
 0.090909       52
 2.695187       51
-1.909091       50
-0.909091       49
-1.241044       48
 29.363636      46
-2.241044       44
 30.363636      43
-0.241044     

2018-07-06 08:04:20,523 logger 29 [INFO]    [make_npy] 0.000000     544
0.177278     245
0.707107      49
6.579496      35
1.479049      26
5.410238      25
0.947832      25
7.713892      25
11.570838     23
1.414214      21
2.828427      21
2.546713      21
6.883803      21
7.713892      19
6.735660      19
3.982555      19
2.121320      18
7.534704      18
0.529829      18
0.701679      18
7.478237      18
10.245801     17
6.883802      17
3.325522      16
6.688936      15
2.446991      15
5.543473      15
6.883808      14
8.249579      14
3.535534      14
3.408024      14
7.993381      14
7.478226      13
10.040723     13
1.944042      13
2.651149      13
6.694984      13
5.543465      13
7.667763      12
6.735660      12
6.993140      12
0.959275      12
7.512224      12
5.137847      12
7.531442      12
1.479045      11
0.486160      11
3.281949      11
1.025332      11
6.708758      11
            ... 
3.552310       1
7.333947       1
7.619528       1
10.230523      1
9.013372  

In [82]:
' 回数をカラムにもたせる '
ft_name = f'CNT_UNU_{p_id}@'
ft_name = f'CNT_UNU_YEAR_{p_id}@'
cnt_data = unuse_year
cnt_prev = cnt_data.groupby(unique_id)['SK_ID_PREV'].size().reset_index().rename(columns={p_id:ft_name})

cnt = base.merge(cnt_prev, on=unique_id, how='left').fillna(0)
' value_countして、外れ値は目視で外してしまう⇒いけてないので、outlierもってくるｗ '
cnt[ft_name].value_counts()

0.0    350115
1.0      5556
2.0       479
3.0        81
4.0        16
6.0         3
7.0         2
5.0         2
8.0         1
Name: CNT_UNU_YEAR_SK_ID_PREV@, dtype: int64

In [83]:
cnt[ft_name] = cnt[ft_name].clip(0, 4)
# cnt[f'CNT_REF_{p_id}@'] = cnt[f'CNT_REF_{p_id}@'].clip(0, 30)
# cnt[f'CNT_APR_{p_id}@'].value_counts()
# cnt[f'CNT_REF_{p_id}@'].value_counts()
make_npy(logger=logger, ignore_list=ignore_features, result=cnt)

2018-07-06 08:20:51,469 logger 29 [INFO]    [make_npy] 0.0    350115
1.0      5556
2.0       479
3.0        81
4.0        24
Name: CNT_UNU_YEAR_SK_ID_PREV@, dtype: int64 
2018-07-06 08:20:51,469 logger 29 [INFO]    [make_npy] 0.0    350115
1.0      5556
2.0       479
3.0        81
4.0        24
Name: CNT_UNU_YEAR_SK_ID_PREV@, dtype: int64 


In [51]:
prev_apr

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE_x,AMT_ANNUITY_x,AMT_APPLICATION,AMT_CREDIT_x,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE_x,WEEKDAY_APPR_PROCESS_START_x,HOUR_APPR_PROCESS_START_x,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE_x,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,TARGET_x,valid_no_x,is_train_x,is_test_x,DAYS_lastplan_month,DAYS_LAST_DUE_diff_lastplan_month,DAYS_original_last_due,DAYS_FIRST_DUE_diff_DAYS_DECISION@,DAYS_LAST_DUE_1ST_VERSION_diff_DAYS_LAST_DUE@,DAYS_TERMINATION_diff_DAYS_LAST_DUE@,DAYS_term_month,DAYS_term_month_1st,AMT_CREDIT_div_AMT_APPLICATION@,...,HOUR_APPR_PROCESS_START_y,HOUSETYPE_MODE,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE_y,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_INCOME_TYPE,NAME_TYPE_SUITE_y,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE,ORGANIZATION_TYPE,OWN_CAR_AGE,REGION_POPULATION_RELATIVE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,TARGET_y,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START_y,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,is_test_y,is_train_y,valid_no_y
0,2462805,299809,Cash loans,9842.400,90000.000,90000.00,NaN,90000.000,THURSDAY,13,Y,1.0,NaN,NaN,NaN,XNA,Approved,-75.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash Street: high,-75.0,-74.0,-63.0,-63.0,-63.0,0.0,0,3.0,1,0,-65.855889,2.855889,-63.0,1.0,0.0,0.0,12.0,12.0,1.000000,...,15,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Cash loans,Secondary / secondary special,Separated,House / apartment,Working,Unaccompanied,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Laborers,Government,NaN,0.031329,2,2,0,0,0,0,0,NaN,NaN,SUNDAY,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3.0
1,1004367,299809,Cash loans,10660.860,135000.000,152820.00,NaN,135000.000,THURSDAY,9,Y,1.0,NaN,NaN,NaN,XNA,Approved,-67.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,24.0,high,Cash Street: high,NaN,-66.0,-43.0,-43.0,-43.0,1.0,0,3.0,1,0,-52.665322,9.665322,-43.0,1.0,0.0,0.0,24.0,24.0,1.132000,...,15,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Cash loans,Secondary / secondary special,Separated,House / apartment,Working,Unaccompanied,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Laborers,Government,NaN,0.031329,2,2,0,0,0,0,0,NaN,NaN,SUNDAY,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3.0
2,1460450,299809,Cash loans,9222.390,76500.000,81549.00,NaN,76500.000,WEDNESDAY,9,Y,1.0,NaN,NaN,NaN,XNA,Approved,-52.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash X-Sell: high,NaN,-51.0,-40.0,-40.0,-39.0,1.0,0,3.0,1,0,-43.157498,3.157498,-40.0,1.0,0.0,1.0,12.0,12.0,1.066000,...,15,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Cash loans,Secondary / secondary special,Separated,House / apartment,Working,Unaccompanied,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Laborers,Government,NaN,0.031329,2,2,0,0,0,0,0,NaN,NaN,SUNDAY,NaN,NaN,NaN,NaN,NaN,NaN,0,1,3.0
3,1129168,299809,Cash loans,4921.200,45000.000,45000.00,NaN,45000.000,THURSDAY,7,Y,1.0,NaN,NaN,NaN,XNA,Approved,-80.0,XNA,XAP,NaN,Repeater,XNA,Cash,x-sell,Country-wide,NaN,Consumer electronics,12.0,high,Cash Street: high,NaN,-

In [60]:
app_list = ['app_AMT_CREDIT', 'app_AMT_ANNUITY', 'app_AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'app_AMT_GOODS_PRICE', 'a_EXT_SOURCE_avg@']

# dataset = prev_ref.copy()
# dataset = prev_ref_year.copy()
dataset = prev_apr.copy()
base_columns = list(dataset.columns)
prefix = 'p_APREND_'
# prefix = 'p_APREND_YEAR_'
# prefix = 'p_APREND_HALFYEAR_'
# ref_list = amt_list.remove('cnt_annuity')

for col_1 in amt_list:
    for col_2 in app_list:
        dataset = division_feature(df=dataset, first=col_1, second=col_2)
        
make_features = list(dataset.columns)
for col in base_columns:
    make_features.remove(col)

select_level_agg(base=base, data=dataset, level=level, num_list=make_features, method_list=method_list, ignore_list=ignore_features, prefix=prefix)

2018-07-06 20:42:35,465 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_CREDIT_div_AMT_ANNUITY@
method: sum
['SK_ID_CURR'] 
2018-07-06 20:42:35,465 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_CREDIT_div_AMT_ANNUITY@
method: sum
['SK_ID_CURR'] 
../../../github/module/feature_engineering.py:26: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  result = df.groupby(level)[feature].agg({'tmp': {method}})
2018-07-06 20:42:36,713 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_CREDIT_div_AMT_ANNUITY@_sum@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:36,713 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_CREDIT_div_AMT_ANNUITY@_sum@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:36,841 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07

2018-07-06 20:42:37,226 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_CREDIT_div_AMT_ANNUITY@
method: max
['SK_ID_CURR'] 
2018-07-06 20:42:37,226 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_CREDIT_div_AMT_ANNUITY@
method: max
['SK_ID_CURR'] 
2018-07-06 20:42:37,344 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_CREDIT_div_AMT_ANNUITY@_max@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:37,344 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_CREDIT_div_AMT_ANNUITY@_max@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:37,426 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:42:37,426 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:42:37,467 logger 29 [INFO]    [make_npy] 100.000000    216
200.000000    175
20.023588     133
40.000000     118
50.000000

2018-07-06 20:42:37,779 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_CREDIT_div_AMT_ANNUITY@
method: std
['SK_ID_CURR'] 
2018-07-06 20:42:37,779 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_CREDIT_div_AMT_ANNUITY@
method: std
['SK_ID_CURR'] 
2018-07-06 20:42:37,894 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_CREDIT_div_AMT_ANNUITY@_std@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:37,894 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_CREDIT_div_AMT_ANNUITY@_std@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:37,975 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:42:37,975 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:42:37,999 logger 29 [INFO]    [make_npy] 0.000000      479
75.116560       2
6.635253        2
100.164834      2
49.559377

2018-07-06 20:42:38,268 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_ANNUITY_div_AMT_ANNUITY@
method: mean
['SK_ID_CURR'] 
2018-07-06 20:42:38,268 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_ANNUITY_div_AMT_ANNUITY@
method: mean
['SK_ID_CURR'] 
2018-07-06 20:42:38,367 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_ANNUITY_div_AMT_ANNUITY@_mean@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:38,367 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_ANNUITY_div_AMT_ANNUITY@_mean@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:38,435 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:42:38,435 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:42:38,483 logger 29 [INFO]    [make_npy] 0.300354     33
1.001179     25
1.527199     16
1.166975     14
1.080973 

2018-07-06 20:42:38,794 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_ANNUITY_div_AMT_ANNUITY@
method: min
['SK_ID_CURR'] 
2018-07-06 20:42:38,794 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_ANNUITY_div_AMT_ANNUITY@
method: min
['SK_ID_CURR'] 
2018-07-06 20:42:38,911 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_ANNUITY_div_AMT_ANNUITY@_min@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:38,911 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_ANNUITY_div_AMT_ANNUITY@_min@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:42:38,988 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:42:38,988 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:42:39,024 logger 29 [INFO]    [make_npy] 0.840991    144
0.300354    129
0.811156    105
0.500590     93
1.001179     

...

2018-07-06 20:43:38,530 logger 29 [INFO]    [make_npy] 0.000000         275
33750.000000     241
22500.000000     201
11250.000000     122
49500.000000      97
44442.887658      83
24750.000000      79
74250.000000      76
49500.000000      68
99000.000000      65
33332.165744      64
45000.000000      62
88885.775316      58
55553.609573      53
13500.000000      51
22221.443829      49
74250.000000      49
6750.000000       46
24750.000000      45
31500.000000      44
18000.000000      41
22725.000000      38
19800.000000      38
99000.000000      36
51750.000000      35
25875.000000      35
29700.000000      34
24750.000000      33
11925.000000      33
67500.000000      33
20250.000000      32
66664.331487      31
27000.000000      31
15750.000000      30
26703.025328      30
23175.000000      29
54000.000000      28
27000.000000      28
53406.050657      27
123750.000000     26
14850.000000      26
77775.053402      23
36000.000000      23
90000.000000      23
23625.000000      23


2018-07-06 20:43:38,996 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:43:38,996 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:43:39,040 logger 29 [INFO]    [make_npy] 33750.000000     307
22500.000000     260
11250.000000     153
24750.000000     137
49500.000000     129
74250.000000     116
44442.887658     113
45000.000000     109
49500.000000      95
99000.000000      93
33332.165744      81
67500.000000      79
13500.000000      78
74250.000000      75
24750.000000      75
55553.609573      70
25875.000000      68
31500.000000      64
22221.443829      63
29700.000000      61
88885.775316      60
6750.000000       60
51750.000000      57
27652.500000      57
19800.000000      57
22725.000000      54
99000.000000      53
90000.000000      52
26703.025328      52
53406.050657      51
56250.000000      50
27000.000000      49
54000.000000      49
18000.000000      48
27000.000000      46
36000.000000      46
24750

2018-07-06 20:43:39,316 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_GOODS_PRICE_div_cnt_annuity@
method: std
['SK_ID_CURR'] 
2018-07-06 20:43:39,316 logger 125 [INFO]    [select_level_agg] 
num: app_AMT_GOODS_PRICE_div_cnt_annuity@
method: std
['SK_ID_CURR'] 
2018-07-06 20:43:39,449 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_GOODS_PRICE_div_cnt_annuity@_std@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:43:39,449 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'app_AMT_GOODS_PRICE_div_cnt_annuity@_std@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:43:39,554 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:43:39,554 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:43:39,597 logger 29 [INFO]    [make_npy] 0.000000e+00    161
1.969690e+02      6
1.909188e+04      5
2

2018-07-06 20:43:39,903 logger 125 [INFO]    [select_level_agg] 
num: cnt_annuity_div_a_EXT_SOURCE_avg@@
method: mean
['SK_ID_CURR'] 
2018-07-06 20:43:39,903 logger 125 [INFO]    [select_level_agg] 
num: cnt_annuity_div_a_EXT_SOURCE_avg@@
method: mean
['SK_ID_CURR'] 
2018-07-06 20:43:40,020 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'cnt_annuity_div_a_EXT_SOURCE_avg@@_mean@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:43:40,020 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'cnt_annuity_div_a_EXT_SOURCE_avg@@_mean@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:43:40,115 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:43:40,115 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:43:40,157 logger 29 [INFO]    [make_npy] 14.012888    1
16.225918    1
17.613648    1
17.603900    1
11.51

2018-07-06 20:43:40,440 logger 125 [INFO]    [select_level_agg] 
num: cnt_annuity_div_a_EXT_SOURCE_avg@@
method: min
['SK_ID_CURR'] 
2018-07-06 20:43:40,440 logger 125 [INFO]    [select_level_agg] 
num: cnt_annuity_div_a_EXT_SOURCE_avg@@
method: min
['SK_ID_CURR'] 
2018-07-06 20:43:40,561 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'cnt_annuity_div_a_EXT_SOURCE_avg@@_min@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:43:40,561 logger 129 [INFO]    [select_level_agg] 
base: Index(['SK_ID_CURR'], dtype='object')
result: Index(['SK_ID_CURR', 'cnt_annuity_div_a_EXT_SOURCE_avg@@_min@['SK_ID_CURR']'], dtype='object') 
2018-07-06 20:43:40,648 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:43:40,648 logger 132 [INFO]    [select_level_agg] 
result shape: (356255, 2) 
2018-07-06 20:43:40,758 logger 29 [INFO]    [make_npy] 9.714941      1
10.880011     1
19.188194     1
18.851383     1
17.75